In [1]:
import numpy as np
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

import wfdb
from utils import qrs_detect, comp_cosEn, save_dict
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [3]:
final = pd.read_pickle("final_test_30SEC.pkl")

In [4]:
final[['Sequence Label']] = final[['Sequence Label']].apply(lambda col:pd.Categorical(col).codes)

,Sequence Number,Sequence Label,AF Burden,Signal Length,Chunked Label,y_pred,AF Episodes,Predicted AF Burden
0,979,persistent atrial fibrillation,0.999999,872215,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",135.0,0.928670
1,980,paroxysmal atrial fibrillation,0.036434,35159,"[0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.000000
2,981,non atrial fibrillation,0.000000,50946,"[0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.000000
3,982,persistent atrial fibrillation,0.999994,163946,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",27.0,0.988130
4,983,persistent atrial fibrillation,0.999992,122069,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",20.0,0.983051


In [12]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(final['AF Burden'],final['Predicted AF Burden'])

0.15734215366414325

In [32]:
conditions = [
    (final['Predicted AF Burden'] > .7),
    ((final['Predicted AF Burden'] > 0.2) & (final['Predicted AF Burden'] < 0.7)),
    (final['Predicted AF Burden'] < 0.2)
    ]
choices = ['2', '1', '0']
final['Predicted Sequence Label'] = np.select(conditions, choices, default='0')
final.head()

,Sequence Number,Sequence Label,AF Burden,Signal Length,Chunked Label,y_pred,AF Episodes,Predicted AF Burden,Predicted Seuqence Label,Predicted Sequence Label
0,979,2,0.999999,872215,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",135.0,0.928670,2,2
1,980,1,0.036434,35159,"[0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.000000,0,0
2,981,0,0.000000,50946,"[0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.0,0.000000,0,0
3,982,2,0.999994,163946,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",27.0,0.988130,2,2
4,983,2,0.999992,122069,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",20.0,0.983051,2,2


In [30]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(final['Sequence Label'],final['Predicted Sequence Label'])

0.3532219570405728